In [13]:
import torch
import time
import numpy as np
import torch.optim as optim
import torch.nn as nn
from torch.utils import data
import random
import math
import matplotlib.pyplot as plt
import scipy.signal
from scipy.interpolate import make_interp_spline, BSpline
import scipy.interpolate as interpolate
import os
import random
from os import listdir
from os.path import isfile, join
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import scipy.signal
from collections import Counter
from pdb import set_trace as st
from matplotlib.colors import hsv_to_rgb
import torch.nn.functional as F

In [28]:
# Load data and stuff
data_y = []
print(len(listdir('./data_polar')))
index = 0
ii = 0

j = []
print(listdir('./data_polar'))
for f in listdir('./data_polar'):
    index+=1
#     print(index)
    
    label = f.split('_')[0].split('c')[1]
    data_y.append(int(label)-1)
    if int(label)-1 == 10:
        j.append(ii)
        
    ii+=1

data_y = np.array(data_y)



15716
['c10_959.npy', 'c1_1184.npy', 'c6_92.npy', 'c9_358.npy', 'c3_292.npy', 'c3_1252.npy', 'c5_551.npy', 'c10_826.npy', 'c11_428.npy', 'c3_4710.npy', 'c10_1000.npy', 'c3_4918.npy', 'c5_704.npy', 'c3_2830.npy', 'c1_361.npy', 'c3_6550.npy', 'c7_177.npy', 'c8_331.npy', 'c6_415.npy', 'c3_2904.npy', 'c3_5334.npy', 'c3_2900.npy', 'c5_1005.npy', 'c6_1359.npy', 'c3_3356.npy', 'c2_307.npy', 'c3_5495.npy', 'c4_403.npy', 'c6_1312.npy', 'c7_193.npy', 'c5_845.npy', 'c6_473.npy', 'c4_20.npy', 'c3_3301.npy', 'c3_606.npy', 'c3_3045.npy', 'c3_3919.npy', 'c10_103.npy', 'c11_354.npy', 'c3_6579.npy', 'c5_199.npy', 'c3_4159.npy', 'c3_1054.npy', 'c3_5314.npy', 'c3_5610.npy', 'c11_939.npy', 'c11_1090.npy', 'c3_2446.npy', 'c1_607.npy', 'c3_1287.npy', 'c7_458.npy', 'c11_612.npy', 'c3_3937.npy', 'c3_4909.npy', 'c3_3307.npy', 'c11_1036.npy', 'c2_187.npy', 'c8_180.npy', 'c3_895.npy', 'c11_194.npy', 'c3_1413.npy', 'c10_1363.npy', 'c7_475.npy', 'c3_3920.npy', 'c3_1504.npy', 'c2_75.npy', 'c7_286.npy', 'c3_30.npy',

In [31]:
train, test = index_split(True)
print(Counter(data_y[train]))
print(Counter(data_y[test]))
print(len(train))
print(len(test))
print(len(train)+len(test))
train, test = index_split(False)
print(Counter(data_y[train]))
print(Counter(data_y[test]))
print(len(train))
print(len(test))
print(len(train)+len(test))

Counter({2: 2578, 5: 538, 9: 528, 0: 491, 10: 459, 4: 456, 6: 233, 7: 228, 8: 203, 3: 181, 1: 164})
Counter({2: 2768, 5: 605, 9: 585, 0: 541, 10: 487, 4: 475, 8: 247, 7: 232, 6: 231, 1: 194, 3: 186})
6059
6551
12610
Counter({2: 1156, 5: 234, 9: 232, 0: 229, 4: 218, 10: 218, 6: 113, 7: 92, 8: 91, 3: 84, 1: 80})
Counter({2: 1511, 5: 322, 9: 315, 0: 299, 10: 271, 4: 243, 7: 134, 8: 132, 6: 123, 1: 101, 3: 97})
2747
3548
6295


In [ ]:
# Load data and stuff
data_x = []
data_y = []
print(len(listdir('./data_polar')))
index = 0
for f in listdir('./data_polar'):
    index+=1
#     print(index)
    data = np.load(join('./data_polar', f))
    label = f.split('_')[0].split('c')[1]
    data_x.append(data)
    data_y.append(int(label)-1)

data_x = np.array(data_x)
data_y = np.array(data_y)

xshape = data_x.shape
data_x = data_x.reshape((xshape[0], xshape[1], 1, xshape[2], xshape[3]))
data_x[:, 0,...] = np.arccos(data_x[:, 0, 0,...]).reshape(data_x[:, 0,...].shape)
data_x[:, 1,...] = data_x[:, 4,...]
data_x = data_x[:, :2,...]



In [16]:
def index_split(use_1517):
    #Splitting method for our MSTAR data
    #If use_1517 is True, use the 15/17 depression split
    #If use_1517 is False, use the Seen/Unseen data split
    
    csv_path = './chipinfo.csv' 
    df = pd.read_csv(csv_path)
    training = df.loc[df['depression'] == 17]
    subclass_9 = training.loc[training['target_type'] != 'bmp2_tank']
    subclass_8 = subclass_9.loc[subclass_9['target_type'] != 't72_tank'].index.values
    class_1_train = np.array(training.loc[training['serial_num']=='c21'].index.values)
    class_3_train = np.array(training.loc[training['serial_num']=='132'].index.values)
    subclass = np.concatenate([subclass_8, class_1_train, class_3_train], axis=0)
    training = training.index
    testing = df.loc[df['depression'] == 15]
    
    subclass_test9 = testing.loc[testing['target_type'] != 'bmp2_tank']
    subclass_test8 = np.array(subclass_test9.loc[subclass_test9['target_type']=='t72_tank'].index.values)
    class_1_test2 = np.array(testing.loc[testing['serial_num']=='9563'].index.values)
    class_1_test3 = np.array(testing.loc[testing['serial_num']=='9566'].index.values)
    class_3_test2 = np.array(testing.loc[testing['serial_num']=='812'].index.values)
    class_3_test3 = np.array(testing.loc[testing['serial_num']=='s7'].index.values)
    subclass_test = np.concatenate([subclass_test8, class_1_test2, class_1_test3, class_3_test2, class_3_test3], axis=0)
    testing = np.array(testing.index.values)
    
    if use_1517:
        
        return training, testing
    else:
        
        return subclass, subclass_test

In [34]:
a = torch.tensor([0, 0, 1, 1, 1, 1])
b = torch.tensor([2, 2, 2, 2, 2, 2])
print(a)
print(b)
b[a > 0] = 999
print(b)

torch.randperm(20)

tensor([0, 0, 1, 1, 1, 1])
tensor([2, 2, 2, 2, 2, 2])
tensor([  2,   2, 999, 999, 999, 999])


tensor([19, 15,  2, 17, 13,  7,  0, 18,  4,  6,  5,  9, 10, 11, 12,  3,  1,  8,
        14, 16])

In [12]:
print(index_split(True)[0].shape)
print(index_split(True)[1].shape)

print(index_split(False)[0].shape)
print(index_split(False)[1].shape)

> <ipython-input-11-1a2d8c4d56ac>(14)index_split()
-> subclass = np.concatenate([subclass_8, class_1_train, class_3_train], axis=0)
(Pdb) ee = subclass_9 = training.loc[training['target_type'] == 'bmp2_tank']
(Pdb) ff = ee.loc[ee['serial_num']!='c21']
(Pdb) ff
      target_TgtNontgt target_type  ClutterType  num_rows  num_columns  \
1365                 1   bmp2_tank            0       128          128   
1366                 1   bmp2_tank            0       128          128   
1367                 1   bmp2_tank            0       128          128   
1368                 1   bmp2_tank            0       128          128   
1369                 1   bmp2_tank            0       128          128   
1370                 1   bmp2_tank            0       128          128   
1371                 1   bmp2_tank            0       128          128   
1372                 1   bmp2_tank            0       128          128   
1373                 1   bmp2_tank            0       128          128   

(Pdb) training.loc[training['target_type'] == 'clutter']
Empty DataFrame
Columns: [target_TgtNontgt, target_type, ClutterType, num_rows, num_columns, azimuth, depression, version, configuration, FSImageNumber, Column, Row, NormalizedScore, Mean, Variance, StandardDeviation, RMS, Skewness, Kurtosis, Maximum, TotalIntegral, ZeroValuedPoints, serial_num, filename]
Index: []

[0 rows x 24 columns]
(Pdb) subclass.loc[subclass['target_type'] == 'clutter']
*** NameError: name 'subclass' is not defined
(Pdb) exiy
*** NameError: name 'exiy' is not defined
(Pdb) exit


BdbQuit: 

In [ ]:
def sort_samples(x, y, per_class_count, num_class):
    # Gets *per_class_count* instances of num_class 
    output = {}
    for classes in range(num_class):
        output[classes] = []
        current_count = 0
        for indices in range(len(y)):
            if y[indices] == classes:
                output[classes].append(x[indices])
                current_count += 1
            if current_count == per_class_count:
                break
    for classes in range(num_class):
        output[classes] = np.array(output[classes])
    return output

In [ ]:
def sort_samples_from_source(per_class_count, num_class):
    # Gets *per_class_count* instances of num_class 
    output = {}
    
    for classes in range(num_class):
        
        output[classes] = []
        current_count = 0
            
            
        for f in listdir('./data_polar'):
            label = f.split('_')[0].split('c')[1]
            
            if int(label)-1 == classes:
                
                data = np.load(join('./data_polar', f))
                output[classes].append(data)
                current_count += 1
                if current_count == per_class_count:
                    break
                    
        
        
        
                        
                        
    
    
        
        
        
    for classes in range(num_class):
        output[classes] = np.array(output[classes])
        
        xshape = output[classes].shape
        data_x = output[classes].reshape((xshape[0], xshape[1], 1, xshape[2], xshape[3]))
        data_x[:, 0,...] = np.arccos(data_x[:, 0, 0,...]).reshape(data_x[:, 0,...].shape)
        data_x[:, 1,...] = data_x[:, 4,...]
        data_x = data_x[:, :2,...]
        output[classes] = data_x

    return output

In [ ]:
some_samples = sort_samples_from_source(5, 11)

In [ ]:
some_samples[0].shape

In [ ]:
f = plt.figure(figsize=(20,10))
print(some_samples[0][0].shape)

for i in range(1, 6):
    f.add_subplot('15'+str(i))
    x0 = some_samples[0][i-1]
    
    # 2, 1, H, W 
#     x0 = (x0 - np.mean(x0))/ np.std(x0)
    x0 = (x0 - np.min(x0, keepdims=True)) / (np.max(x0, keepdims=True) - np.min(x0, keepdims=True))
    print(x0.shape)
    
    
    img = x0[:, 0, :, :]
    img = img.transpose((1, 2, 0))
    img = np.insert(img, 1, 1, axis=2)
    
    # For 
    img[:, :, 2] = np.sqrt(img[:, :, 2]) # 
    img[:, :, 2] = (img[:, :, 2] - np.min(img[:, :, 2], keepdims=True)) / (np.max(img[:, :, 2], keepdims=True) - np.min(img[:, :, 2], keepdims=True))
    
    rgb = hsv_to_rgb(img)
    plt.imshow(rgb)

In [ ]:
def visualize_output(img):
    # Image is shape 2, 1, H, W
    img = img.detach().numpy()
    print(img.shape)
    img = (img - np.min(img, keepdims=True)) / (np.max(img, keepdims=True) - np.min(img, keepdims=True))
    img = img.transpose((1, 2, 0))
    img = np.insert(img, 1, 1, axis=2)
    img[:, :, 2] = np.sqrt(img[:, :, 2])
    img[:, :, 2] = (img[:, :, 2] - np.min(img[:, :, 2], keepdims=True)) / (np.max(img[:, :, 2], keepdims=True) - np.min(img[:, :, 2], keepdims=True))
    
    from matplotlib.colors import hsv_to_rgb
    rgb = hsv_to_rgb(img)
    
    plt.imshow(rgb)
#     plt.imshow(img[:, :, 2])
    return

In [ ]:
class ComplexConv(nn.Module):
    def __init__(self, in_channels, out_channels, kern_size, stride=(1, 1), num_tied_block=1, padding=0, dilation=1, groups=1, use_log=False):
        
        ## Magnitude do log / exp; Phase do weighted 
        super(ComplexConv, self).__init__()
        
        self.kern_size = kern_size
        self.stride = stride
        
        self.use_log = use_log
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        self.num_blocks = num_tied_block
        
        if in_channels % num_tied_block != 0:
            assert("Number of tied block convolution needs to be multiple of in_channels: "+str(in_channels))
            
        if  out_channels % num_tied_block != 0:
            assert("Number of tied block convolution needs to be multiple of out_channels: "+str(out_channels))
        
        self.in_channels = in_channels // num_tied_block
        self.out_channels = out_channels // num_tied_block
        
        if type(kern_size) == int:
            self.kern_size = (kern_size, kern_size)
        
        self.eps = 1e-7 # For stability issues in logarithm calculation    
        
        # Initialize kernels
        self.mag_kernel = torch.nn.Parameter(torch.rand((self.out_channels, self.in_channels, self.kern_size[0], self.kern_size[1])), requires_grad=True)
        self.phase_kernel = torch.nn.Parameter(torch.rand((self.out_channels, self.in_channels, self.kern_size[0], self.kern_size[1])), requires_grad=True)
        
        # Consistent with torch's initialization
        n = self.in_channels
        for k in self.kern_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        
        self.mag_kernel.data.uniform_(-stdv, stdv)
        self.phase_kernel.data.uniform_(-stdv, stdv)
        
        
    def forward(self, x):
        # Input is of shape [Batch, 2, channel, height, width]
        single_input = x # 0 is phase, 1 is magnitude
        
        eps = self.eps
        phase = single_input[:, 0, ...]
        mag = single_input[:, 1, ...]
        
        if self.use_log:
            log_mag = torch.log(mag+eps)
        else:
            log_mag = mag

        mag_kernel_sqrd = self.mag_kernel ** 2
        mag_kernel = mag_kernel_sqrd / torch.sum(torch.sum(mag_kernel_sqrd, dim=2, keepdim=True), dim=3, keepdim=True)
        
        mag_list = [F.conv2d(log_mag[:, i*self.in_channels:(i+1)*self.in_channels, ...], \
                             mag_kernel, None, self.stride, self.padding, self.dilation, \
                             self.groups) for i in range(self.num_blocks)]
        
        log_output = torch.cat(mag_list, dim=1)
        
        if self.use_log:
            mag_output = torch.exp(log_output)
        else:
            mag_output = log_output

        phase_kernel_sqrd = self.phase_kernel ** 2
        phase_kernel = phase_kernel_sqrd / torch.sum(torch.sum(phase_kernel_sqrd, dim=2, keepdim=True), dim=3, keepdim=True)
        
        phase_list = [F.conv2d(phase[:, i*self.in_channels:(i+1)*self.in_channels, ...], \
                             phase_kernel, None, self.stride, self.padding, self.dilation, \
                             self.groups) for i in range(self.num_blocks)]
        phase_output = torch.cat(phase_list, dim=1)

        final_output = torch.cat([phase_output.unsqueeze(1), mag_output.unsqueeze(1)], dim=1)
        return final_output

In [ ]:
print(s.shape)
# Euclidean experiment
euclidean_layer = ComplexConv(10, 20, (3, 3), 1, num_tied_block=2)
single_input = s
print("Input shape: "+str(single_input.shape))
s = euclidean_layer(single_input)
print("Output shape: "+str(s.shape))

In [ ]:
"""
Utility function for computing output of convolutions
takes a tuple of (h,w) and returns a tuple of (h,w)
"""
def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    if type(stride) is not tuple:
        stride = (stride, stride)
    h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride[0]) + 1)
    w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride[1]) + 1)
    return h, w


class Iterative_wFM(nn.Module):
    
    def __init__(self, in_channels, kernel_size, stride=(1, 1), padding=0, dilation=1, initialize_test=False):
        super(Iterative_wFM, self).__init__()
        
        # actual_kern_size stores the size of the kernel, in one integer
        
        if type(kernel_size) == int:
            actual_kernel_size = kernel_size * kernel_size
            kernel_size = (kernel_size, kernel_size)
        else:
            actual_kernel_size = kernel_size[0] * kernel_size[1]
        
        
        if initialize_test:
            self.mag_kernel = torch.nn.Parameter(torch.ones((in_channels, actual_kernel_size)), requires_grad=True)
#             self.phase_kernel = torch.nn.Parameter(torch.rand((channel)), requires_grad=True)
            self.bias = torch.tensor(50)
        
        else:
            self.mag_kernel = torch.nn.Parameter(torch.rand((in_channels, actual_kernel_size)), requires_grad=True)
            self.bias = torch.nn.Parameter(torch.rand((in_channels)), requires_grad=True)
#             self.phase_kernel1 = torch.nn.Parameter(torch.rand((channel)), requires_grad=True) #maybe try this
        
        self.in_channels = in_channels
        self.actual_kern_size = actual_kernel_size
        self.kern_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        
    
        
    def apply_kernel(self, inputs):
        # Input of shape [Batch, 2, channel, kernel_size, #blocks]
        # Applies the kernel to the inputs, expect same shape as the weights 
        mag_kernel_weights = self.mag_kernel / torch.cumsum(self.mag_kernel, dim=1) #compute successive weights, should be same dimension as kernel
        bias = self.bias
        # The wFM is calculated along the **kernel_size** dimension where 
        # the first entry would be m1 in documentation

        # I noticed that the wFM is always in the form of mag/phase so there is no need to represent it in cartesian form
        first_mean = inputs[:, :, :, 0, :] # [Batch, 2, #channel, #blocks]
        x_mag = torch.sqrt(torch.sum(first_mean**2, dim=1)) # [Batch, #channel, #blocks]
        x_phase = torch.atan2(first_mean[:, 1, ...], first_mean[:, 0, ...])
        
        x_mag = torch.transpose(x_mag, 1, 2) # [Batch, #blocks, #channels]
        x_phase = torch.transpose(x_phase, 1, 2) # [Batch, #blocks, #channels]
        
        for i in range(1, self.actual_kern_size):
            current_input = inputs[:, :, :, i, :] # [Batch, 2, # channel, #blocks]
            y_mag = torch.sqrt(torch.sum(current_input**2, dim=1)) # [Batch, #channel, #blocks]
            y_phase = torch.atan2(current_input[:, 1, ...], current_input[:, 0, ...]) # [Batch, #channel, #blocks]
            
            y_mag = torch.transpose(y_mag, 1, 2) # [Batch, #blocks, #channels]
            y_phase = torch.transpose(y_phase, 1, 2) # [Batch, #blocks, #channels]

            t = mag_kernel_weights[:, i] # [#channel]
            theta = torch.acos(x_phase * y_phase) # [Batch, #blocks, #channels]


            new_mag = x_mag * (((y_mag + bias)/(x_mag + bias)) ** t) #- bias # - bias
            new_phase = 1 / torch.sin(theta) * (torch.sin(t * theta) * x_phase + torch.sin((1 - t) * theta) * y_phase)


            x_mag = new_mag
            x_phase = new_phase
            
        final_x = x_mag * torch.cos(x_phase).unsqueeze(1) # [Batch, 1, #blocks, #channels]
        final_y = x_mag * torch.sin(x_phase).unsqueeze(1) # [Batch, 1, #blocks, #channels]
        
        final_x = torch.transpose(final_x, 2, 3)
        final_y = torch.transpose(final_y, 2, 3)
        
        # Output of shape [Batch, 2, #channels, #blocks]
        return torch.cat((final_x, final_y), dim=1)
    
    def forward(self, x):
        # Input is [B, 2, channel, H, W]
        x_shape = x.shape
        # Since unfold only deals with 2d inputs, combine complex (2) with channel
        x = x.reshape(x_shape[0], x_shape[1]*x_shape[2], x_shape[3], x_shape[4])
        
        fold = nn.Unfold(kernel_size=self.kern_size, dilation=self.dilation, padding=self.padding, stride=self.stride)
        folded = fold(x)
        fold_shape = folded.shape
        
        
        # Reshape into [Batch, 2, channel, kernel_size, #blocks]
        folded = folded.reshape(x_shape[0], x_shape[1], x_shape[2], self.actual_kern_size, fold_shape[-1])
        
        # Now is [Batch, 2, #channels, #blocks]
        applied_output = self.apply_kernel(folded)
        final_shape = applied_output.shape
        applied_output = applied_output.reshape(final_shape[0], final_shape[1]*final_shape[2], final_shape[3])
        
        next_shape = conv_output_shape(x_shape[-2:], kernel_size=self.kern_size, dilation=self.dilation, pad=self.padding, stride=self.stride)
        folder = nn.Fold(next_shape, kernel_size=(1,1))
        
        result = folder(applied_output)
        result_shape = result.shape
        return result.reshape((x_shape[0], x_shape[1], x_shape[2], result_shape[-2], result_shape[-1]))
        

In [ ]:
c = torch.ones(5, 2, 3)
d = torch.ones(5, 3, 2)

In [ ]:
torch.matmul(c, d).shape

In [ ]:
a=5
b=3
print(int((a-b)/2))
print(int((a-b)/2)+b)


In [ ]:
# Complex experiment
complex_layer = Iterative_wFM(1, (3, 3))
single_input = torch.tensor(some_samples[0][2]).unsqueeze(0).float()

single_input = torch.rand(1, 2, 5, 100, 100)


print("Input shape: "+str(single_input.shape))
s = complex_layer(single_input)
print("Output shape: "+str(s.shape))

In [ ]:
f = plt.figure(figsize=(20,10))
for i in range(1, 6):
    f.add_subplot('15'+str(i))
    x0 = s[0, :, i-1, :, :]
    visualize_output(x0)

In [ ]:
# Complex experiment
complex_layer = ComplexConv(1, 5, (3, 3), 1)
single_input = torch.tensor(some_samples[0][2]).unsqueeze(0).float()
print("Input shape: "+str(single_input.shape))
s = complex_layer(single_input)
print("Output shape: "+str(s.shape))

In [ ]:
f = plt.figure(figsize=(20,10))
for i in range(1, 6):
    f.add_subplot('15'+str(i))
    x0 = s[0, :, i-1, :, :]
    visualize_output(x0)

In [ ]:
class EuclideanConv(nn.Module):
    def __init__(self, in_channels, out_channels, kern_size, stride=(1, 1), padding=0, dilation=1, groups=1):
        super(EuclideanConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kern_size = kern_size
        self.stride = stride
        
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        
        if type(kern_size) == int:
            self.kern_size = (kern_size, kern_size)
         
        # Only need one kernel. wEM(x, y)
        self.mag_kernel = torch.nn.Parameter(torch.rand((out_channels, in_channels, self.kern_size[0], self.kern_size[1])), requires_grad=True)
        self.phase_kernel = torch.nn.Parameter(torch.rand((out_channels, in_channels, self.kern_size[0], self.kern_size[1])), requires_grad=True)
        
        # Consistent with torch's initialization
        n = self.in_channels
        for k in self.kern_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        
        self.mag_kernel.data.uniform_(-stdv, stdv)
        self.phase_kernel.data.uniform_(-stdv, stdv)
    
    def forward(self, x):
        # Input is of shape [B, 2, c, h, w]
        single_input = x # 0 is phase, 1 is magnitude
        
        phase = single_input[:, 0, ...]
        mag = single_input[:, 1, ...]

        
        mag_output = F.conv2d(mag, self.mag_kernel, None, self.stride, self.padding, self.dilation, self.groups)

        phase_output = F.conv2d(phase, self.phase_kernel, None, self.stride, self.padding, self.dilation, self.groups)

        final_output = torch.cat([phase_output.unsqueeze(1), mag_output.unsqueeze(1)], dim=1)
        return final_output

In [ ]:
# Euclidean experiment
euclidean_layer = EuclideanConv(1, 5, (3, 3), 1)
single_input = torch.tensor(some_samples[0][2]).unsqueeze(0).float()
print("Input shape: "+str(single_input.shape))
s = euclidean_layer(single_input)
print("Output shape: "+str(s.shape))

In [ ]:
# Euclidean experiment
euclidean_layer = EuclideanConv(5, 10, (3, 3), 1)
single_input = s
print("Input shape: "+str(single_input.shape))
s = euclidean_layer(single_input)
print("Output shape: "+str(s.shape))

In [ ]:
f = plt.figure(figsize=(20,10))
for i in range(1, 6):
    f.add_subplot('15'+str(i))
    x0 = s[0, :, i-1, :, :]
    visualize_output(x0)

In [ ]:
class AverageConv(nn.Module):
    def __init__(self, in_channels, out_channels, kern_size, stride=(1, 1), padding=0, dilation=1, groups=1):
        super(AverageConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kern_size = kern_size
        self.stride = stride
        
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        
        if type(kern_size) == int:
            self.kern_size = (kern_size, kern_size)
         
        self.mag_kernel = torch.nn.Parameter(torch.ones((out_channels, in_channels, self.kern_size[0], self.kern_size[1]))/ torch.sum(torch.sum(torch.ones((out_channels, in_channels, self.kern_size[0], self.kern_size[1])), dim=2, keepdim=True), dim=3, keepdim=True), requires_grad=True)
        self.phase_kernel = torch.nn.Parameter(torch.ones((out_channels, in_channels, self.kern_size[0], self.kern_size[1]))/ torch.sum(torch.sum(torch.ones((out_channels, in_channels, self.kern_size[0], self.kern_size[1])), dim=2, keepdim=True), dim=3, keepdim=True), requires_grad=True)

    def forward(self, x):
        # Input is of shape [B, 2, c, h, w]
        single_input = x # 0 is phase, 1 is magnitude
        
        phase = single_input[:, 0, ...]
        mag = single_input[:, 1, ...]

        
        
        mag_output = F.conv2d(mag, self.mag_kernel, None, self.stride, self.padding, self.dilation, self.groups)

        phase_output = F.conv2d(phase, self.phase_kernel, None, self.stride, self.padding, self.dilation, self.groups)

        final_output = torch.cat([phase_output.unsqueeze(1), mag_output.unsqueeze(1)], dim=1)
        return final_output

In [ ]:
# Average experiment
average_layer = AverageConv(1, 5, (3, 3), 1)
single_input = torch.tensor(some_samples[0][2]).unsqueeze(0).float()
print("Input shape: "+str(single_input.shape))
s = average_layer(single_input)
print("Output shape: "+str(s.shape))

In [ ]:
f = plt.figure(figsize=(20,10))
for i in range(1, 6):
    f.add_subplot('15'+str(i))
    x0 = s[0, :, i-1, :, :]
    visualize_output(x0)

In [ ]:
class DistanceTransform(nn.Module):
    def __init__(self, in_channels, kern_size, log_var=True, padding=0, dilation=1, groups=1):
        
        ## Magnitude do log / exp; Phase do weighted 
        ## log_var == True if magnitude already taken **log**
        
        super(DistanceTransform, self).__init__()
        
        self.take_log = log_var
        self.in_channels = in_channels
        self.kern_size = kern_size
        
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        
        if type(kern_size) == int:
            self.kern_size = (kern_size, kern_size)
        
        self.eps = 1e-7 # For stability issues in logarithm calculation    
        
        # Initialize kernels
        self.mag_kernel = torch.nn.Parameter(torch.rand((in_channels, in_channels, \
                                                         self.kern_size[0], self.kern_size[1])), requires_grad=True)
        self.phase_kernel = torch.nn.Parameter(torch.rand((in_channels, in_channels, \
                                                           self.kern_size[0], self.kern_size[1])), requires_grad=True)
        
        # Consistent with torch's initialization
        n = self.in_channels
        for k in self.kern_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        
        self.mag_kernel.data.uniform_(-stdv, stdv)
        self.phase_kernel.data.uniform_(-stdv, stdv)
        
        # For final weighted sum between mag and phase
        self.weight = torch.nn.Parameter(torch.rand(2), requires_grad=True)
        
    def forward(self, x):
        # Input is of shape [Batch, 2, channel, height, width]
        single_input = x # 0 is phase, 1 is magnitude
        x_shape = x.shape
        
        eps = self.eps
        phase = single_input[:, 0, ...]
        mag = single_input[:, 1, ...]
        
#       This is implemented with prior log scaling
        if not self.take_log:
            log_mag = torch.log(mag+eps)
        else:
            log_mag = mag

        mag_kernel_sqrd = self.mag_kernel ** 2
        mag_kernel = mag_kernel_sqrd / torch.sum(torch.sum(mag_kernel_sqrd, dim=2, \
                                                           keepdim=True), dim=3, keepdim=True)
        
        ## Conv2d takes strides the same as kernels, as of now.
        log_output = F.conv2d(log_mag, mag_kernel, None, self.kern_size, self.padding, \
                              self.dilation, self.groups)

        mag_output = torch.exp(log_output)

        phase_kernel_sqrd = self.phase_kernel ** 2
        phase_kernel = phase_kernel_sqrd / torch.sum(torch.sum(phase_kernel_sqrd, \
                                                               dim=2, keepdim=True), dim=3, keepdim=True)
        
        ## Conv2d takes strides the same as kernels, as of now.
        phase_output = F.conv2d(phase, phase_kernel, None, self.kern_size, self.padding, \
                                self.dilation, self.groups)
        
#         # Need to do normalization, but on which dimension???
#         phase_output = phase_output / 
#         final_output = torch.cat([phase_output.unsqueeze(1), mag_output.unsqueeze(1)], dim=1)
    
        upsampler = nn.Upsample(size=x_shape[-2:], mode='nearest')
        
        # Upsample the respective outputs to the same size as inputs
        mag_output = upsampler(mag_output)
        phase_output = upsampler(phase_output)
        
        # Take magnitude distance
        if self.take_log: 
            mag_distance = torch.abs(torch.log(mag/(mag_output+eps)))
        else:
            mag_distance = torch.abs(torch.log(torch.exp(mag)/(mag_output+eps)))
            
        phase_distance = torch.abs(phase-phase_output)
        
        
        
    
        return mag_distance * (self.weight[0] **2) + phase_distance * (self.weight[1] **2)
    

In [ ]:
# Average experiment
average_layer = DistanceTransform(1, (2, 2), False)
single_input = torch.tensor(some_samples[0][2]).unsqueeze(0).float()
print("Input shape: "+str(single_input.shape))
s = average_layer(single_input)
print("Output shape: "+str(s.shape))

plt.imshow(s[0, 0, ...].detach().numpy())  

In [ ]:
m = nn.Upsample(size=(4, 9), mode='nearest')
m(a)

In [ ]:
c = nn.functional.fold(b, (4, 6), kernel_size=(1,1), stride=(1, 1))
print(c.shape)
print(c[0, 0, :, :])